In [1]:
# import necessary libraries
import os, glob, sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import multiprocessing as mp

from tqdm.auto import tqdm
from ipywidgets import FloatProgress
from scipy import ndimage
from PIL import Image
from importlib import reload
from functools import partial

# append path
sys.path.append('../')

# custom
from utils.Metrics import iou
from utils.Traits import area_calc
from utils.GetFileNames import get_file_names
import utils.SegmentImage as SegmentImage
import utils.Measure as Measure
import utils.Preprocessing as Preprocessing


/mnt/DGX01/Personal/krusepi/.venv/lib/python3.9/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.16). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
# directories
seg_path = '../data/pop_scale/pop_scale_segmentations/'
in_path = '../data/test/test_images/'
postproc_path = '../data/pop_scale/pop_scale_segmentations_postproc/'
out_path = '../data/pop_scale/pop_scale_segmentations_by_pod/'
save_path = '../data/pop_scale/pop_scale_segmentations_measurements/'

# get file names
seg_names = get_file_names(seg_path, '.png')

In [3]:
def postprocess_multi(seg_img, out_path):
    # get image
    seg = np.array(Image.open('../data/pop_scale/pop_scale_segmentations/' + seg_img)) / 255

    # remove artifacts
    post_seg = Preprocessing.remove_artifacts(seg)

    # convert to 
    post_seg = Image.fromarray((post_seg * 255).astype(np.uint8))
    post_seg.save(postproc_path + seg_img)

In [4]:
with mp.Pool(mp.cpu_count()) as pool:
        r = tqdm(pool.imap(partial(postprocess_multi, out_path=out_path), seg_names),
                total = len(seg_names))
        for _ in r:
                pass

  0%|          | 0/767 [00:00<?, ?it/s]

In [8]:
reload(Measure)
Measure.measure_pods(pred_path = postproc_path,
                     input_path = in_path,
                     pod_save_path = out_path,
                     measurement_save_path = save_path,
                     image_names = seg_names,
                     plot=False,
                     verbose=False)

Measuring Pods...


  0%|          | 0/767 [00:00<?, ?it/s]

IndexError: boolean index did not match indexed array along dimension 0; dimension is 477 but corresponding boolean dimension is 517